In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

<ipython-input-1-3a2706ca3b87>:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import json
import glob
import numpy as np
import pandas as pd
import pickle as pkl
import os
pd.set_option("display.max_rows", None)

In [3]:
accents = [
    "assamese_female_english",
    "manipuri_female_english",
    "gujarati_female_english",
    "malayalam_male_english",
    "tamil_male_english",
    "hindi_male_english",
    "rajasthani_male_english",
    "kannada_male_english"  
]
print(f"length of accent-list = {len(accents)}")

length of accent-list = 8


In [4]:
from collections import Counter
from functools import reduce


def kernels(kernel_string):
    # kernel_string = kernel_string[len("kernel_"):]
    parts = kernel_string.split('_')
    return {
        "ground kernel": parts[0].split('=')[-1],
        "ground query kernel": parts[1].split('=')[-1],
        "query query kernel": parts[2].split('=')[-1],
    }


def get_duration(path, speaker_accent):
    with open(path, 'r') as file:
        lines = [json.loads(line.strip()) for line in file.readlines()]
    accents = [line["audio_filepath"].split('/')[-4] for line in lines]
    durations = [line["duration"] for line in lines]
    total_duration = np.sum(durations)
    target_duration = reduce(lambda x,y: x+y, [duration for (duration, accent) in zip(durations, accents) if accent == speaker_accent], 0)
    duration_str = f"{target_duration}/{total_duration}"
    return duration_str
    
    


def get_counts(path):
    with open(path, 'r') as file:
        cnter = Counter([json.loads(line.strip())["audio_filepath"].split('/')[-4] for line in file.readlines()])
    return cnter

In [5]:



budget = 150
target = 20
dict_lst = []

for accent in accents:
#     glob_pattern = f"{accent}/all/budget_{budget}/target_{target}/FL1MI_*/*/*/*/*/*/train.json"
    glob_pattern = f"{accent}/all/budget_{budget}/target_{target}/FL1MI_*/*/*/*/*/*/train.json"
    lst = glob.glob(glob_pattern)
    lst.extend(glob.glob(glob_pattern.replace("FL1MI", "LogDMI")))
    for pth in lst:
        path_comp = os.path.normpath(pth).split(os.sep)
        dct = {
            "accent": path_comp[0],
            "budget": path_comp[2].split('_')[-1],
            "target": path_comp[3].split('_')[-1],
            "fxn": path_comp[4].split('_')[0],
            "etaScale": path_comp[4].split('_')[-1],
            "accent_features": path_comp[5][len("accent_"):],
            "content_features": path_comp[6][len("content_"):],
            "accent_similarity": path_comp[8][len("accent_"):],
            "content_similarity": path_comp[9][len("content_"):],
            "accent_counts": get_counts(pth),
            "duration_frac": get_duration(pth, accent)
        }
        dct.update((kernels(path_comp[7][len("kernel_"):])))
        dict_lst.append(dct)
    

In [6]:
df = pd.DataFrame(dict_lst)
filtered_df = df[
    (df["accent_features"] == "39_norm") & 
    (df["ground kernel"] == "content") & 
    (df["ground query kernel"] == "accent") & 
    (df["query query kernel"] == "content")
].sort_values(["accent", "etaScale"], ascending=True).reset_index(drop=True)
filtered_df
# [["accent", "budget", "target", "etaScale", "duration_frac", "accent_counts"]].head(150)

,accent,budget,target,fxn,etaScale,accent_features,content_features,accent_similarity,content_similarity,accent_counts,duration_frac,ground kernel,ground query kernel,query query kernel
0,assamese_female_english,150,20,FL1MI,0.05,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content
1,assamese_female_english,150,20,FL1MI,0.1,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content
2,assamese_female_english,150,20,LogDMI,0.1,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 11, 'manipuri_fema...",38.630604/736.45038,content,accent,content
3,assamese_female_english,150,20,FL1MI,0.2,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content
4,assamese_female_english,150,20,LogDMI,0.2,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 68, 'manipuri_fema...",253.108145/734.9910179999999,content,accent,content
5,assamese_female_english,150,20,FL1MI,0.3,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content
6,assamese_female_english,150,20,LogDMI,0.3,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 77, 'manipuri_fema...",282.956935/735.3757420000002,content,accent,content
7,assamese_female_english,150,20,FL1MI,0.4,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content
8,assamese_female_english,150,20,LogDMI,0.4,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 84, 'manipuri_fema...",304.6428939999999/734.95675,content,accent,content
9,assamese_female_english,150,20,FL1MI,0.5,39_norm,tf_idf_2gram,cosine,cosine,"{'assamese_female_english': 143, 'kannada_male...",727.7610850000002/735.429939,content,accent,content


In [ ]:
filtered_df.to_csv("mod_kernel_selection_stats.csv", index=False)